Lists all GPCC netcdf files

In [1]:
data_directory = '/gws/pw/j05/cop26_hackathons/bristol/project02/data/obs/GPCC/full_data_monthly_v2020/05'

!ls {data_directory}

full_data_monthly_v2020_1891_1900_05.nc
full_data_monthly_v2020_1901_1910_05.nc
full_data_monthly_v2020_1911_1920_05.nc
full_data_monthly_v2020_1921_1930_05.nc
full_data_monthly_v2020_1931_1940_05.nc
full_data_monthly_v2020_1941_1950_05.nc
full_data_monthly_v2020_1951_1960_05.nc
full_data_monthly_v2020_1961_1970_05.nc
full_data_monthly_v2020_1971_1980_05.nc
full_data_monthly_v2020_1981_1990_05.nc
full_data_monthly_v2020_1991_2000_05.nc
full_data_monthly_v2020_2001_2010_05.nc
full_data_monthly_v2020_2011_2019_05.nc
readme_md5_05.txt


ncdump one of these netcdf files

In [2]:
!ncdump -h {data_directory}/full_data_monthly_v2020_1891_1900_05.nc

netcdf full_data_monthly_v2020_1891_1900_05 {
dimensions:
	lon = 720 ;
	lat = 360 ;
	time = UNLIMITED ; // (120 currently)
variables:
	double lon(lon) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
	double lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
	double time(time) ;
		time:standard_name = "time" ;
		time:units = "hours since 1891-1-1 00:00:00" ;
		time:calendar = "proleptic_gregorian" ;
		time:axis = "T" ;
	float precip(time, lat, lon) ;
		precip:long_name = "gpcc full data monthly product version 2020, precipitation per grid" ;
		precip:units = "mm/month" ;
		precip:code = 20 ;
		precip:_FillValue = -99999.99f ;
		precip:missing_value = -99999.99f ;
	float numgauge(time, lat, lon) ;
		numgauge:long_name = "gpcc full data monthly product version 2020, number of stations per grid" ;
		numgauge:units = "gauges per gridcell" ;
		n

Reads all netcdfs into xarray

In [4]:
import xarray as xr
dataset = xr.open_mfdataset(data_directory + '/*.nc')

In [5]:
dataset

<xarray.Dataset>
Dimensions:                       (lat: 360, lon: 720, time: 1548)
Coordinates:
  * lon                           (lon) float64 -179.8 -179.2 ... 179.2 179.8
  * lat                           (lat) float64 89.75 89.25 ... -89.25 -89.75
  * time                          (time) datetime64[ns] 1891-01-01 ... 2019-1...
Data variables:
    precip                        (time, lat, lon) float32 dask.array<chunksize=(120, 360, 720), meta=np.ndarray>
    numgauge                      (time, lat, lon) float32 dask.array<chunksize=(120, 360, 720), meta=np.ndarray>
    infilled_numgauges            (time, lat, lon) float32 dask.array<chunksize=(120, 360, 720), meta=np.ndarray>
    interpolation_error           (time, lat, lon) float32 dask.array<chunksize=(120, 360, 720), meta=np.ndarray>
    interpolation_error_infilled  (time, lat, lon) float32 dask.array<chunksize=(120, 360, 720), meta=np.ndarray>
    diff_new_old_method           (time, lat, lon) float32 dask.array<chunksize=(120, 360, 720), meta=np.ndarray>
Attributes: (12/20)
    CDI:                       Climate Data Interface version 1.7.0 (http://m...
    Conventions:               CF-1.4
    history:                   Sun Dec 13 23:00:21 2020: cdo -setgatts,gattfi...
    title:                     GPCC Full Data Monthly Product Version 2020, p...
    summary:                   The Full Data Monthly Product is of much highe...
    keywords:                  precipitation climatology,gpcc,global,gpcp,
    ...                        ...
    time_coverage_resolution:  month
    geospatial_lat_min:        -90.
    geospatial_lat_max:        90.
    geospatial_lon_min:        -180.
    geospatial_lon_max:        180.
    CDO:                       Climate Data Operators version 1.7.0 (http://m...

Cuts out data in spatial domain and calculates zonal mean

In [19]:
data_cutout = dataset.sel(lat=slice(25, -25), lon=slice(20, 30)).mean(dim='lon')

In [20]:
data_cutout

<xarray.Dataset>
Dimensions:                       (lat: 100, time: 1548)
Coordinates:
  * lat                           (lat) float64 24.75 24.25 ... -24.25 -24.75
  * time                          (time) datetime64[ns] 1891-01-01 ... 2019-1...
Data variables:
    precip                        (time, lat) float32 dask.array<chunksize=(120, 100), meta=np.ndarray>
    numgauge                      (time, lat) float32 dask.array<chunksize=(120, 100), meta=np.ndarray>
    infilled_numgauges            (time, lat) float32 dask.array<chunksize=(120, 100), meta=np.ndarray>
    interpolation_error           (time, lat) float32 dask.array<chunksize=(120, 100), meta=np.ndarray>
    interpolation_error_infilled  (time, lat) float32 dask.array<chunksize=(120, 100), meta=np.ndarray>
    diff_new_old_method           (time, lat) float32 dask.array<chunksize=(120, 100), meta=np.ndarray>

Output to a netcdf

In [21]:
data_cutout.to_netcdf(path='pr_1m_zonAvgNikulin_GPCC_1891-2019_05.nc')

Test the netcdf

In [23]:
!ncdump -h ./pr_1m_zonAvgNikulin_GPCC_1891-2019_05.nc

netcdf test {
dimensions:
	lat = 100 ;
	time = 1548 ;
variables:
	double lat(lat) ;
		lat:_FillValue = NaN ;
		lat:standard_name = "latitude" ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
	int64 time(time) ;
		time:standard_name = "time" ;
		time:axis = "T" ;
		time:units = "days since 1891-01-01 00:00:00" ;
		time:calendar = "proleptic_gregorian" ;
	float precip(time, lat) ;
		precip:_FillValue = NaNf ;
	float numgauge(time, lat) ;
		numgauge:_FillValue = NaNf ;
	float infilled_numgauges(time, lat) ;
		infilled_numgauges:_FillValue = NaNf ;
	float interpolation_error(time, lat) ;
		interpolation_error:_FillValue = NaNf ;
	float interpolation_error_infilled(time, lat) ;
		interpolation_error_infilled:_FillValue = NaNf ;
	float diff_new_old_method(time, lat) ;
		diff_new_old_method:_FillValue = NaNf ;
data:

 lat = 24.75, 24.25, 23.75, 23.25, 22.75, 22.25, 21.75, 21.25, 20.75, 20.25, 
    19.75, 19.25, 18.75, 18.25, 17.75, 17.25, 16.75, 16.25, 15.75

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

